## Analysing the experimental data of  [Brydges et al, Science 2019](https://doi.org/10.1126/science.aau4963)

In [1]:
using RandomMeas
using ProgressMeter
using Revise

In [2]:
## parameters 
N = 10 # System size
times = [0,1,2,3,4,5] # quench times in ms
ntimes = length(times)  # number of time points
ξ = siteinds("Qubit", N); # site indices of the shadows

# Parameters specifying the Renyi entropies of interest
n = 6 # maximal Renyi index (n-th Renyi entropy)
NAmax = 6; #maximal subsystem size. We always consider subsystems of the form [1,2,...,NA].

We now load the experimental randomized measurement data obtained in Brydges et al., Science 2018.

In [3]:
data = Vector{MeasurementData}(undef, ntimes)

for s in 1:ntimes
    data[s] = import_measurement_data("BrydgesScience2019data/measurement_data_10_T_" * string(times[s]) * ".npz",site_indices=ξ, add_value = 1)
end 

We estimate the purity using the direct purity estimation formula presented Brydges et al., Science 2019 and using classical shadows Huang et al., Nat. Phys. 2020.

In [4]:
p_direct = zeros(Float64,ntimes,NAmax)
@showprogress for s in 1:ntimes
    for NA in 1:NAmax
        p_direct[s,NA] = get_purity_direct(data[s],collect(1:NA))
    end
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:09


In [5]:
@show p_direct[2,:]

p_direct[2, :] = [0.7548595973154374, 0.6805170469798665, 0.6481793288590602, 0.618093959731544, 0.6069744966442955, 0.6067140939597319]


6-element Vector{Float64}:
 0.7548595973154374
 0.6805170469798665
 0.6481793288590602
 0.618093959731544
 0.6069744966442955
 0.6067140939597319

We estimate higher order Renyi entropies (up to order n) using batch shadows (c.f. Rath et al., PRXQ 2023).

In [6]:
#Tr(rho^n) and Renyis from batch shadows
p_bshadow = zeros(Float64,ntimes,NAmax,n-1)
S_bshadow = zeros(Float64,ntimes,NAmax,n-1)
@showprogress for s in 1:ntimes
    for NA in 1:NAmax
        uA = [ut[1:NA] for ut in u[s]]
        ξA = ξ[1:NA]
        ρs = get_batch_shadows(data[s,:,:,1:NA],ξA,uA,n)
        p_bshadow[s,NA,:] = get_moments(ρs, ξA, collect(2:n))
        for nt in 2:n
            S_bshadow[s,NA,nt-1]= log2(p_bshadow[s,NA,nt-1])/(1-nt)
        end
    end
end

UndefVarError: UndefVarError: `u` not defined

Finally, we load quantum states generated by classical simulations (master equation evolution) of the experiment, including a decoherence model. We compute Renyi entropies.

In [ ]:
using MAT

ρ = Vector{ITensor}()
p = zeros(Float64,ntimes,NAmax,n-1)
S = zeros(Float64,ntimes,NAmax,n-1)
SvN = zeros(Float64,ntimes,NAmax)
ITensors.disable_warn_order()

for s in 1:ntimes
    qstate = matread("BrydgesScience2019data/rho_10_XY_10_-1.00_0.00"*string(times[s])*"_1_1_1_flr_1.mat")["rho"]
    qstate = reshape(qstate,tuple((2*ones(Int,2*N))...))
    push!(ρ,ITensor(qstate,vcat(ξ',ξ)))
    for NA in 1:NAmax
        A = collect(1:NA)
        ρA = copy(ρ[s])
        for i in 1:N
            if !(i in A)
                ρA *= δ(ξ[i],ξ[i]') 
            end
        end
        for nt in 2:n
            p[s,NA,nt-1] = real(trace(power(ρA,nt),ξ[1:NA]))
            S[s,NA,nt-1]= log2(p[s,NA,nt-1])/(1-nt)
        end
    end
end

In [ ]:
@show p[2,1:NAmax,1]

We plot the purity as a function of NA for the different times.

In [ ]:
using PyPlot

clf()

cm = get_cmap(:viridis)
for s in 1:ntimes
    plot(1:NAmax,p[s,:,1],"--",color=cm(s/ntimes))
    plot(1:NAmax,p_direct[s,:],"o",color=cm(s/ntimes))
    plot(1:NAmax,p_shadow[s,:],"x",color=cm(s/ntimes))
    plot(1:NAmax,p_bshadow[s,:,1],"*",color=cm(s/ntimes))
end
#yscale("log")
xlabel("Partition")
ylabel("Purity")
legend(["theory","hamming","shadow","batch_shadow"])
title("Purity for different times")

gcf()

We plot the higher order Renyi entropies as a function of Renyi index n for a fixed subystem size and various times.

In [ ]:
using PyPlot

clf()
cm = get_cmap(:viridis)
NA = 6
for s in 1:1
    plot(2:n,p[s,NA,:],"--",color=cm(s/ntimes),label="theory")
    plot(2:n,p_bshadow[s,NA,:],"x",color=cm(s/ntimes),label="batch_shadow")
end
for s in 2:ntimes
    plot(2:n,p[s,NA,:],"--",color=cm(s/ntimes))
    plot(2:n,p_bshadow[s,NA,:],"x",color=cm(s/ntimes))
end
xlabel("n")
ylabel(L"p_n(\rho_A)")
legend(title = "NA= $NA ")

gcf()

In [ ]:
using MAT
ITensors.disable_warn_order()
using Base.Threads: @threads

using LinearAlgebra  # For BLAS thread management

# Print the number of Julia threads
println("Number of Julia threads: $(Threads.nthreads())")

# Set BLAS threads to 1
BLAS.set_num_threads(1)
println("Number of BLAS threads: $(BLAS.get_num_threads())")

# Parameters
NU, NM = 500, 150
export_dir = "MeasurementDataExports/"  # Directory to save exported files

# Ensure the export directory exists
isdir(export_dir) || mkpath(export_dir)

for s in [1,3,5]
    # Load and process the quantum state
    qstate = matread("BrydgesScience2019data/rho_10_XY_10_-1.00_0.00"*string(times[s])*"_1_1_1_flr_1.mat")["rho"]
    qstate = reshape(qstate, tuple((2 * ones(Int, 2 * N))...))
    rho = ITensor(qstate, vcat(ξ', ξ))
    rho = MPO(rho, ξ, cutoff=1e-10)

    @threads for sample in 1:100
        # Generate measurement settings and simulate results
        @time measurement_settings = RandomMeas.LocalUnitaryMeasurementSettings(N, NU, site_indices=ξ)
        @time measurement_data = simulate_RandomMeas(rho, measurement_settings, NM, "dense")

        # Construct a meaningful filename
        filename = joinpath(export_dir, "simulated_measurement_data_t_$(times[s])_sample_$(sample)_NU_$(NU)_NM_$(NM).npz")

        # Export the data
        export_measurement_data(measurement_data, filename)
        println("Exported: $filename")

        # Load the exported data

        if sample == 1
            loaded_data = import_measurement_data(filename, site_indices=ξ)

            # Check that the loaded data is the same as the original data
            @assert isapprox(measurement_data.measurement_results, loaded_data.measurement_results)
            @assert isapprox(measurement_data.measurement_settings.local_unitaries, loaded_data.measurement_settings.local_unitaries)

            for NA in 1:NAmax
                @show get_purity_direct(measurement_data,collect(1:NA))
            end
            
        end

    end
end

In [ ]:
using MAT
ITensors.disable_warn_order()
using Base.Threads: @threads

using LinearAlgebra  # For BLAS thread management

# Print the number of Julia threads
println("Number of Julia threads: $(Threads.nthreads())")

# Set BLAS threads to 1
BLAS.set_num_threads(1)
println("Number of BLAS threads: $(BLAS.get_num_threads())")

# Parameters
NU, NM = 500, 150
export_dir = "/Users/aelben/Data/RandomMeas_SampleBrydges/MeasurementDataExports/"  # Directory to save exported files


k_max = 6

p_bshadow = zeros(Float64,100,3,NAmax,k_max-1)
S_bshadow = zeros(Float64,100,3,NAmax,k_max-1)

for i in 1:3

    @threads for sample in 1:100

        @show i, sample

        filename = joinpath(export_dir, "simulated_measurement_data_t_$(times[2*i])_sample_$(sample)_NU_$(NU)_NM_$(NM).npz")

        loaded_data = import_measurement_data(filename, site_indices=ξ)

        local_unitaries = loaded_data.measurement_settings.local_unitaries
        data = loaded_data.measurement_results

        for NA in 1:NAmax
            uA = [local_unitaries[r, 1:NA] for r in 1:size(local_unitaries, 1)]
            ξA = ξ[1:NA]
            ρs = get_batch_shadows(data[:,:,1:NA],ξA,uA,k_max)
            p_bshadow[sample,i, NA,:] = get_moments(ρs, ξA, collect(2:k_max))
            for nt in 2:n
                S_bshadow[sample,i,NA,nt-1]= log2(p_bshadow[sample,i,NA,nt-1])/(1-nt)
            end
        end

    end
end

In [ ]:
using Statistics

In [ ]:
@show size(p_bshadow)
@show size(p)

In [ ]:
p_bshadow_mean = mean(p_bshadow,dims=1)[1,:,:,:]
p_select = p[[2,4,6],:,:]
S_select = S[[2,4,6],:,:]

In [ ]:
@show maximum(abs.(p_select.-p_bshadow_mean))

@show 1/sqrt(500*100)

In [ ]:
@show cov(p_bshadow[:,1,],dims=1)

In [ ]:
using NPZ
npzwrite("sampled_data.npz",p_bshadow=p_bshadow,p=p_select,S=S_select,S_bshadow=S_bshadow,times=times[[2,4,6]],NU=NU,NM=NM,N=N,NAmax=NAmax,k_max=k_max,samples  = samples)